In [1]:
#-----------------------------------------
# Problem Set 1: BLP Estimation Code
#   Tanya Rajan and George Vojta
#
# (Additional thanks to Michael Cuna, 
#  Nadia Lucas and Jordan Rosenthal-Kay 
#  for their helpful comments)
#-----------------------------------------


# importing packages
import numpy as np
import pandas as pd
import sys
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
import jax
import scipy

np.random.seed(0)

In [2]:
# read in data
data = pd.read_csv("psetOne.csv")

In [4]:

# d17 data
d17 = data[data['Market'] == 17].copy()
d2 = data[data['Market'] ==2].copy()


In [5]:
# ------------------------
#       Question 9
# ------------------------

# function calculating choice probabilities per individual
def pchoose(uvec):
    numerator = np.exp(uvec)
    denominator = np.sum(numerator)
    return numerator / (1+denominator)

# function to calculate ubar per individual
def umean2(δ, σ, X, ζ, J):
    util = δ.T + np.dot(X*ζ, σ)
    return np.array(util).reshape((1,J))

def umean(δ, σ, X, ζ, J):
    util = np.zeros(J)
    for j in range(J):
        util[j] = (δ[:,j] + np.dot(X[j,:], (ζ*σ.T).T))
    return util

#(delta1[:,1] + np.matmul(Xvec[2,:], zeta[1,:])* sigma1).T

#np.matmul(Xvec[2,:], (zeta[1,:]* sigma1.T).T)


In [7]:
# function to calculate shares
def sHat(δ, X, σ, ζ, I, J, full="no"):
    
    # compute individual choice probs
    choices = np.zeros(I*J).reshape((I,J))
    for i in range(I):
        choices[i,:] = pchoose(umean2(δ, σ, X, ζ[i,:], J))
    
    # integrating over zetas
    share = (1/I)*np.sum(choices, axis=0)
    
    if full == "full":
        return share, choices
    else:
        return share

#### Test Cases ####
Jtest = 3
Itest = 20


# testing the function above (will take out later)
Xvec = np.array(d17[['Price', 'EngineSize', 'Constant', 'SportsBike', 'Brand2', 'Brand3']])[0:Jtest,:]
zeta = np.hstack((np.random.normal(0,1,2*Itest).reshape(Itest,2), np.zeros(4*Itest).reshape(Itest,4)))

# Test case 1
delta1 = np.full((1,Jtest), 0)
sigma1 = np.array([0, 0, 0, 0, 0, 0]).reshape((6,1))
test1 = sHat(delta1, Xvec, sigma1, zeta, 20, Jtest)

# Test case 2
delta2 = np.full((1,Jtest), 20)
delta2[0,0] = 40
sigma2 = np.array([0, 0, 0, 0, 0, 0]).reshape((6,1))
test2 = sHat(delta2, Xvec, sigma2, zeta, Itest, Jtest)


# Test case 3
delta3 = np.full((1,Jtest), 0)
sigma3 = np.array([.1, .1, 0, 0, 0, 0]).reshape((6,1))
Xvec3 = np.zeros(Jtest*6).reshape(Jtest,6)
test3 = sHat(delta3, Xvec3, sigma3, zeta, Itest, Jtest)

# Test case 4 (like 3 but with original values of X)
test4 = sHat(delta3, Xvec, sigma3, zeta, Itest, Jtest)

# Test case 5 
delta5 = np.full((1,Jtest), 2)
delta5[0,0] = 4
test5, probs_out = sHat(delta5, Xvec, sigma2, zeta, Itest, Jtest, "full")

# Display all
print("Test 1: ", test1)
print("Test 2: ",test2)
print("Test 3: ",test3)
print("Test 4: ",test4)
print("Test 5: ", test5)

share_out = test5


Test 1:  [0.25 0.25 0.25]
Test 2:  [9.99999996e-01 2.06115361e-09 2.06115361e-09]
Test 3:  [0.25 0.25 0.25]
Test 4:  [0.2495407  0.22995882 0.2362604 ]
Test 5:  [0.77580349 0.10499359 0.10499359]


In [8]:
# ------------------------
#       Question 10
# ------------------------

# contraction mapping method
def s_contraction(s, d_init, maxiter, tol, X, I, J, σ, ζ):
    diff = 1
    d_new = d_init
    counter = 0
    while (diff > tol and counter < maxiter):
        d_old = d_new
        s_hat = sHat(d_old, X, σ, ζ, I, J)
        d_new = d_old + np.log(s) - np.log(s_hat)
        diff = np.max(np.abs(d_new - d_old))
        counter += 1
        #if (counter % 50 == 0): # print every 50 iterations
           # print(counter)
    return d_new, counter, diff

# testing the contraction mapping against test case 2
shares = share_out
d_guess = np.full((1, Jtest), .5)
tolerance = 1e-14

s_contraction(shares, d_guess, 10000, tolerance, Xvec, Itest, Jtest, sigma1, zeta)


(array([[4., 2., 2.]]), 1949, 9.769962616701378e-15)

In [9]:
# ------------------------
#       Question 11
# ------------------------

# parsing vector theta into parameters
def parse_theta(θ):
    λ = θ[0:6].reshape(6,1)
    σ = θ[6:].reshape(6,1)
    return λ, σ


# function to calculate xi (making this modular since we need it for Q12)
def get_xi(θ, I, J, X, s, ζ, d_init, intol, dreturn="no"):
    
    # parse
    λ, σ = parse_theta(θ)
    
    # getting an estimate of δ
    d_star, cnt, diff = s_contraction(s, d_init, 1e20, intol, X, I, J, σ, ζ)
    
    # Plugging this in to get ξ
    ξ = np.subtract(d_star.reshape(J,1), np.dot(X, λ))
    if dreturn == "dstar":
        return ξ, d_star
    else:
        return ξ


def objective(θ, ξ, Z, W):
    
    # parsing theta
    λ, σ = parse_theta(θ)
    
    # objective function
    bread = np.dot(Z.T, ξ)
    return np.dot(np.dot(bread.T, W), bread)

#### Test Cases ####
Jtest2 = 8
Itest = 20

Xvec2 = np.array(data[['Price', 'EngineSize', 'Constant', 'SportsBike', 'Brand2', 'Brand3']])[0:Jtest2,:]
inst2 = np.array(data[['z1', 'z2', 'z3', 'z4']])[0:Jtest2,:]
Zvec2 = np.hstack((Xvec2, inst2))
theta = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
weight = np.linalg.inv(np.dot(Zvec2.T, Zvec2))
d_guess2 = np.full((1,Jtest2), 1)
shares = np.array(data[['shares']])[0:Jtest2,:].reshape(1, Jtest2)
intol_test = 1e-20

xi_test2 = get_xi(theta, Itest, Jtest2, Xvec2, shares, zeta, d_guess2, intol_test)
objective(theta, xi_test2, Zvec2, weight)


array([[59.19794556]])

In [14]:
# ------------------------
#       Question 12
# ------------------------

# compute gradient
def gradient(Jy, Z, W, ξ):
    Jg = np.dot(Z.T, Jy)
    g = np.dot(Z.T, ξ)
    grad = 2*np.dot(np.dot(Jg.T, W), g)
    return grad

# compute xi jacobian    
def jacobian_y(X, σ, λ, ζ, δ, I, J):
    # get s_hat, choices
    shat, choices = sHat(δ, X, σ, ζ, I, J, "full")
    
    # first compute J_xi
    J_xi = np.diag((shat)*(1-shat))
    for n in range(shat.size):
        for j in range(shat.size):
            if n != j:
                J_xi[n,j] = -shat[n]*shat[j]
            else: 
                pass

    # second compute J_sigma
    h_grad = np.zeros((J,J,I))
    J_sig_i = np.zeros((J,Xvec.shape[1],I))
    for i in range(I):
        
        # computing gradient of the h function
        h_grad[:,:,i] = np.diag((choices[i,:]*(1-choices[i,:])).flatten())
        for n in range(shat.size):
            for j in range(shat.size):
                if n != j:
                    h_grad[n,j,i] = -choices[i,n]*choices[i,j]
                else: 
                    pass
        
        # computing J_sigma_i for all individuals
        J_sig_i[:,:,i] = np.dot(np.dot(h_grad[:,:,i], X), np.diag(ζ[i,:].flatten()))
    
    # averaging across individuals
    J_sig = np.mean(J_sig_i, axis=2)
    
    # Final J_y
    J_y = np.dot(np.linalg.inv(J_xi), J_sig)
    
    return J_y
    
    
# Testing   
lambda_test, throwaway = parse_theta(theta)
jac_y_test = jacobian_y(Xvec, sigma1, lambda_test, zeta, delta3, Itest, Jtest)
gradient(jac_y_test, Zvec, weight, xi_test).flatten()

In [61]:
# checking against the automatic diff 

def minimization(λ, σ):
    θ = jnp.hstack((λ, σ))
    return objective(θ, jnp.array(xi_test), jnp.array(Zvec), jnp.array(weight))[0,0]

#derivative_fn = grad(minimizer)
lambda_jnp = jnp.array([0.,0.,0.,0.,0., 0.])
sigma_jnp = jnp.array([.1,.2,0.,0.,0.,0.])

grad1_test = jax.grad(minimization, argnums=1)(lambda_jnp, sigma_jnp)

In [16]:
# ------------------------
#       Question 13
# ------------------------
def gmm_optimize(θ_guess, X, Z, s, W, I, J, ζ, d_init, intol):
   
    # setting up important objects
    ξ, d_star = get_xi(θ_guess, I, J, X, s, ζ, d_init, intol, "dstar")
    λ_init, σ_init = parse_theta(θ_guess)
    
    # defining minimizers as fn only of σ
    def minimizer(σ):
        σ = np.array(σ).reshape(6,1)
        θ = np.vstack((λ_init, σ)).flatten()
        return objective(θ, ξ, Z, W)
    
    def min_gradient(σ):
        σ = np.array(σ).reshape(6,1)
        return gradient(jacobian_y(X, σ, λ_init, ζ, d_star, I, J), Z, W, ξ).flatten()
    
    # optimizing
    res = scipy.optimize.minimize(minimizer, σ_init.flatten(), jac=min_gradient, method='Newton-CG')
    σ_star = np.array(res.x).reshape(6,1)
    
    # calculating λ_star
    delta_final, cnt, diff = s_contraction(s, d_init, 50000, intol, X, I, J, σ_star, ζ)
    bread1 = np.dot(X.T, Z)
    term1 = np.linalg.inv(np.dot(np.dot(bread1, W), bread1.T))
    term2 = np.dot(np.dot(np.dot(X.T, Z), W), Z.T)
    λ_star = np.dot(np.dot(term1, term2), delta_final.T)
    
    return np.vstack((λ_star, σ_star)).flatten(), ξ

#### Test Cases ####
Jtest2 = 8
Itest = 20

Xvec2 = np.array(data[['Price', 'EngineSize', 'Constant', 'SportsBike', 'Brand2', 'Brand3']])[0:Jtest2,:]
inst2 = np.array(data[['z1', 'z2', 'z3', 'z4']])[0:Jtest2,:]
Zvec2 = np.hstack((Xvec2, inst2))
theta = np.array([0,0,0,0,0,0,0,0,0,0,0,0])
weight = np.linalg.inv(np.dot(Zvec2.T, Zvec2))
d_guess2 = np.full((1,Jtest2), 1)
shares = np.array(data[['shares']])[0:Jtest2,:].reshape(1, Jtest2)
intol_test = 1e-20

gmm_optimize(theta, Xvec2, Zvec2, shares, weight, Itest, Jtest2, zeta, d_guess2, intol_test)

(array([-1.09062023e+00, -1.72487476e-01,  3.84434109e+00,  1.21303235e+00,
        -5.29539587e+00,  2.31810200e+00, -1.13838657e-13, -3.00664969e-13,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
 array([[ 3.41755771],
        [ 2.21998983],
        [-2.83482062],
        [ 4.31727287],
        [ 1.45962098],
        [ 1.02543285],
        [ 1.53686508],
        [-2.73187448]]))

In [75]:
# 2-step GMM
def gmm_full(θ_guess, X, Z, s, I, J, ζ, d_init, intol):

    ### Weight with identity###
    W = np.eye(Z.shape[1])

    ### Minimize ###
    theta_min, xi = gmm_optimize(θ_guess, X, Z, s, W, I, J, ζ, d_init, intol)
    
    ### Use new xis to calc new W ###
    g = np.dot(Z.T, xi)
    W = (1/Z.shape[0]) * np.dot(g,g.T)

    ### Reminimize and call it a day ###
    theta_min, xi = gmm_optimize(θ_guess, X, Z, s, W, I, J, ζ, d_init, intol)
    
    return theta_min

gmm_full(theta, Xvec2, Zvec2, shares, Itest, Jtest2, zeta, d_guess2, intol_test)

array([ 0.38594288, -0.32258108,  4.30342008, -1.76419601, -4.68074697,
        1.14300231,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ])

In [ ]:
# ---------------------------------
#       Question 13 : Version 2
# ---------------------------------
# The alternative functions here loop through all markets
# to calculate shares, but unfortunately we were unable to get
# this to work. 

# function to extract values of interest from data
def data_extract(df):
    X = np.array(df[['Price', 'EngineSize', 'Constant', 'SportsBike', 'Brand2', 'Brand3']])
    inst = np.array(df[['z1', 'z2', 'z3', 'z4']])
    Z = np.hstack((X, inst))
    s = np.array(df[['shares']]).reshape(1, df.shape[0])
    return X, Z, s
    
# optimization routine for GMM
def gmm_optimize_alt(θ_guess, df, W, I, ζ, intol):
    
    # extract data for whole dataset
    X, Z, s = data_extract(df)
    J = X.shape[0]
    d_init = np.full((1,J),1)
    
    # looping through markets
    ξ = np.array([])
    d_star = np.array([])
    for k in range( df.Market.unique().size ):
        t = k+1
        Xt, Zt, st = data_extract(df[df["Market"] == t].copy())
        Jt = st.size
        d_init_jt = np.full((1,Jt), 1)
        ξ_jt, d_star_jt = get_xi(θ_guess, I, Jt, Xt, st, ζ, d_init_jt, intol, "dstar")
        ξ = np.append(ξ, ξ_jt.flatten())
        d_star = np.append(d_star, d_star_jt.flatten())
        
    # setting up important objects
    #ξ, d_star = get_xi(θ_guess, I, J, X, s, ζ, d_init, intol, "dstar")
    d_star = d_star.reshape(1,J)
    ξ = ξ.reshape(J,1)
    
    λ_init, σ_init = parse_theta(θ_guess)
    
    # defining minimizers as fn only of σ
    def minimizer(σ):
        σ = np.array(σ).reshape(6,1)
        θ = np.vstack((λ_init, σ)).flatten()
        return objective(θ, ξ, Z, W)
    
    def min_gradient(σ):
        σ = np.array(σ).reshape(6,1)
        return gradient(jacobian_y(X, σ, λ_init, ζ, d_star, I, J), Z, W, ξ).flatten()
    
    # optimizing
    res = scipy.optimize.minimize(minimizer, σ_init.flatten(), jac=min_gradient, method='BFGS')
    σ_star = np.array(res.x).reshape(6,1)
    
    #print(d_init.shape, ξ.shape, X.shape, σ_star.shape, ζ.shape, s.shape, I, J)

    # calculating λ_star
    delta_final, c, d = s_contraction(s, d_init, 50000, intol, X, I, J, σ_star, ζ)
    bread1 = np.dot(X.T, Z)
    term1 = np.linalg.inv(np.dot(np.dot(bread1, W), bread1.T))
    term2 = np.dot(np.dot(np.dot(X.T, Z), W), Z.T)
    λ_star = np.dot(np.dot(term1, term2), delta_final.T)
    
    return np.vstack((λ_star, σ_star)).flatten(), ξ

testdata = data[data["Market"] < 2]

#gmm_optimize_alt(theta, testdata, weight, Itest, zeta, intol_test)

# 2-step GMM
def gmm_full(θ_guess, df, I, J, ζ, d_init, intol):
    
    ### Getting vectors of interest in whole data ###
    X, Z, s = data_extract(df)
    
    ### Weight with identity###
    W = np.eye(Z.shape[1])

    ### Minimize ###
    theta_min, xi = gmm_optimize_alt(θ_guess, df, W, I, ζ, intol)
    
    ### Use new xis to calc new W ###
    g = np.dot(Z.T, xi)
    W = (1/Z.shape[0]) * np.linalg.inv(np.dot(g,g.T))

    ### Reminimize and call it a day ###
    theta_min, xi = gmm_optimize_alt(θ_guess, df, W, I, ζ, intol)
    
    return theta_min

gmm_full(theta, testdata, Itest, Jtest, zeta, d_guess, intol_test)